In [ ]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai openai python-dotenv javascript numpy genson
%reload_ext autoreload
%autoreload 2

In [ ]:
# Setup
from dotenv import load_dotenv
import os
from graph.master_graph import create_graph
from IPython.display import Image, display

# Load .env file
load_dotenv('.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")

# Set environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# Create Graph
workflow = create_graph()
app = workflow.compile()
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Define question and invoke app
from langchain_core.messages import HumanMessage

question = """
    Extract all FunctionCalls where receiverId is 'app.nearcrowd.near'. For each function call, I need signerId, block height, receiptId, block datetime, methodName. 
    Also add all fields from args that are decoded from base64-encoded JSON. I only need actions for successful receipts. 
    Use block height 119688212 for parsing the JS code, and only pull data from the last 2 days.
    """

inputs = {
    "messages":[HumanMessage(content=question)],
    "block_schema": "",
    "block_heights":[], # TO DO: Add in block height logging
    "extract_block_data_code":"",
    "table_creation_code":"",
    "data_upsertion_code": "",
    "indexer_logic":"",
    "iterations": 0,
    "error":"",
    "should_continue": False,
    }

# RUN MASTER APP
result = app.invoke(inputs)
table_creation_code = result['table_creation_code'].replace("\\n","\n")
extract_block_data_code = result['extract_block_data_code'].replace("\\n","\n")
data_upsertion_code = result['data_upsertion_code'].replace("\\n","\n")
indexer_logic = result['indexer_logic'].replace("\\n","\n")

In [ ]:
print(f'SQL DDL: {table_creation_code}')
print(f'Data Upsertion Logic: {data_upsertion_code}')